# MultiVector Retriever

In alcuni task potrebbe essere utile utilizzare più vettori per identificare un singolo documento.
LangChain ci mette a disposizione l'oggetto **MultiVectorRetriever** per semplificare l'esecuzione di query che sfruttino i multipli vettori disponibili per rintracciare ogni documento.
Tra i tanti metodi particolari e specifici che potremmo implementare, tre in particolare sono largamente utilizzati:
* **Chunk più piccoli**: dividere un documento in chunk di differenti dimensioni in modo da sfruttare i chunk più piccoli per identificare al meglio la query dell'utente e i chunk più grandi, che contengono quelli piccoli utilizzati per la ricerca, per restituire chunk di testo più grandi di quelli utilizzati per la ricerca (vedi esempio precedente su `ParentDocumentRetriever`).
* **Riepilogo**: creazione di un riepilogo per ciascun documento, incorporato poi insieme al (o al posto del) documento.
* **Query ipotetiche**: creare domande ipotetiche per ogni documento, domande a cui il documento risponderebbe perfettamento, per poi incorporale insieme (o al posto del) documento.

In questo notebook implementeremo gli ultimi due metodi.

------

### Paper scientifici che dimostrano l'efficacia delle tecniche di generazione riepiloghi e query ipotetiche


| Titolo                    | Approccio                      | Fase di generazione                      | Vantaggi principali                                                                                                          |
| ------------------------- | ------------------------------ | ---------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------- |
| *QuOTE*                   | Question-oriented embeddings   | Domande per chunk (indexing)             | Miglior allineamento query‑chunk; alta accuratezza retrieval ([strathweb.com][1], [arXiv][2], [ResearchGate][3], [arXiv][4]) |
| *HyPE*                    | Prompt embedding precomputati  | Domande pre-generate (indexing)          | Precisione+Recall superiore; zero latenza runtime ([SSRN][5], [ResearchGate][6], [strathweb.com][1])                         |
| *HyDE*                    | Risposte ipotetiche a runtime  | Risposta generata all’arrivo della query | Miglior precisione zero‑shot; flessibile su domini sconosciuti ([arXiv][7], [Haystack Documentation][8])                     |
| *Comparativa RAG vs HyDE* | Studio empirico su modelli LLM | Runtime per entrambi                     | Trade‑off tra rilevanza e latenza, RAG più efficiente su edge ([arXiv][9])                                                   |

[1]: https://www.strathweb.com/2025/07/rag-agent-with-hype-pattern-using-semantic-kernel/ "RAG Agent with HyPE Pattern using Semantic Kernel"
[2]: https://arxiv.org/html/2502.10976v1 "QuOTE: Question-Oriented Text Embeddings"
[3]: https://www.researchgate.net/publication/389090378_QuOTE_Question-Oriented_Text_Embeddings "(PDF) QuOTE: Question-Oriented Text Embeddings"
[4]: https://arxiv.org/abs/2502.10976 "QuOTE: Question-Oriented Text Embeddings"
[5]: https://papers.ssrn.com/sol3/papers.cfm?abstract_id=5139335 "Bridging the Question-Answer Gap in Retrieval-Augmented ..."
[6]: https://www.researchgate.net/publication/393738017_Bridging_the_Question-Answer_Gap_in_Retrieval-Augmented_Generation_Hypothetical_Prompt_Embeddings "Bridging the Question-Answer Gap in Retrieval-Augmented ..."
[7]: https://arxiv.org/abs/2212.10496 "Precise Zero-Shot Dense Retrieval without Relevance Labels"
[8]: https://docs.haystack.deepset.ai/docs/hypothetical-document-embeddings-hyde "Hypothetical Document Embeddings (HyDE)"
[9]: https://arxiv.org/abs/2506.21568 "Assessing RAG and HyDE on 1B vs. 4B-Parameter ..."


In [1]:
# caricamento di documenti 

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loaders = [
    TextLoader("data/dialoghi/dialogo.txt"),
    TextLoader("data/dialoghi/dialogo2.txt"),
    TextLoader("data/dialoghi/dialogo3.txt"),
    TextLoader("data/dialoghi/dialogo4.txt"),
    TextLoader("data/dialoghi/dialogo5.txt"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())
text_splitter = RecursiveCharacterTextSplitter(chunk_size=650)
docs = text_splitter.split_documents(docs)

In [2]:
print(len(docs), "chunk creati")

33 chunk creati


## Riepilogo

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

In [4]:
summaries = chain.batch(docs, {"max_concurrency": 5})

In [5]:
for summ in summaries:
    print(summ, "\n\n-------------\n\n")

In a clothing store, a customer asks a sales assistant for help. The customer expresses interest in trying on a pair of black pants displayed in the window and specifies their size as 50. The sales assistant promptly provides the requested pants. 

-------------


In a conversation between a customer and a shop assistant, they discuss the recent weather changes. The customer expresses relief at the cooler weather today, noting that yesterday felt like July. The shop assistant agrees, mentioning how hot it was, and describes changing shirts multiple times due to the heat. They recall a sudden storm that soaked the customer on their way home. The shop assistant remarks on the typical pattern of summer storms following heatwaves, while the customer hopes the pleasant weather holds out as they plan to go downtown later. 

-------------


In a casual conversation between a shopkeeper and a customer, they discuss their shared interest in sports, specifically a recent basketball game between 

In [6]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_classic.storage import InMemoryByteStore
from langchain_classic.retrievers.multi_vector import MultiVectorRetriever
import uuid

vectorstore = Chroma(collection_name="summaries", embedding_function=OpenAIEmbeddings())

# salvataggio dei documenti originali in memoria
store = InMemoryByteStore()

id_key = "doc_id"

retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in docs]

In [7]:
doc_ids

['8eba7ac6-2e56-4cae-ba7a-ec971e62c241',
 '0601f08c-8eef-4f3c-ac01-f5ed0f80988c',
 'd468d9e7-640f-424a-92b5-fdb72871b011',
 '06cf7b53-e88f-41b9-b79d-698b72b5aaf2',
 '70407837-28de-481e-80f4-cb5e5aa94cef',
 'f16ec6c4-4a46-49ed-a30c-1951a8c1d617',
 '8befab80-4a75-4f12-b5f3-389c49dce86e',
 'f1533ceb-f0a1-4c6e-8c15-e2e4c6c99078',
 '1725e07e-0d9b-42f9-8af1-4a1a9a78946e',
 'cf35a1b0-9d12-46ec-9e05-99d9dbc6235e',
 '8519b2f0-9b15-4c31-83d9-cef782186b5c',
 'afa0ffa3-c270-43fb-9508-0572b6c0de63',
 '49b206d4-d843-4bbb-a46b-2dc07182374e',
 '09b781b2-4fe0-44c2-9cc0-81ac85399ecf',
 'd3a3a8d1-d66f-4c7e-a527-6d82a7ff252e',
 '998b0ed0-d753-48f6-a420-565a27cbbe6c',
 'd837b4af-aff3-464d-8af5-04aae5044fef',
 '83080eea-2ee7-4197-ae69-831a84965ea5',
 '62c61a8b-a26f-4ef6-acd3-643e1e700e06',
 'aae0d420-08e9-429c-8308-d1e0c4acb421',
 '13d4e4f2-aef4-405d-9c02-f97214218fab',
 '586bdc04-7ad5-4412-9a32-b832927d76c5',
 '6cf06ef4-6c47-4e27-b578-c9cb3d8b45b9',
 '14da6cfb-5a6d-44ee-95a6-5663e4c6e088',
 'bdf8a5a4-2833-

In [8]:
from langchain_core.documents import Document

# creazione documenti con identificativo univoco nei metadata e sintesi del testo iniziale come contenuto
summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]}) for i, s in enumerate(summaries)
]

In [9]:
summary_docs

[Document(metadata={'doc_id': '8eba7ac6-2e56-4cae-ba7a-ec971e62c241'}, page_content='In a clothing store, a customer asks a sales assistant for help. The customer expresses interest in trying on a pair of black pants displayed in the window and specifies their size as 50. The sales assistant promptly provides the requested pants.'),
 Document(metadata={'doc_id': '0601f08c-8eef-4f3c-ac01-f5ed0f80988c'}, page_content='In a conversation between a customer and a shop assistant, they discuss the recent weather changes. The customer expresses relief at the cooler weather today, noting that yesterday felt like July. The shop assistant agrees, mentioning how hot it was, and describes changing shirts multiple times due to the heat. They recall a sudden storm that soaked the customer on their way home. The shop assistant remarks on the typical pattern of summer storms following heatwaves, while the customer hopes the pleasant weather holds out as they plan to go downtown later.'),
 Document(meta

In [10]:
retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs))) # imposta il valore `docs` as ogni `docs_id`

# # Metodo alternativo:
# # possiamo aggiungere il doc_id ai chunk originali
# for i, doc in enumerate(summary_docs):
#     doc.metadata[id_key] = doc_ids[i]
# retriever.vectorstore.add_documents(summary_docs)

In [11]:
sub_docs = vectorstore.similarity_search("mamme che parlano")

sub_docs

[Document(id='91c504ad-289b-4a47-8256-9ed8eb5354f1', metadata={'doc_id': '49b206d4-d843-4bbb-a46b-2dc07182374e'}, page_content='In una conversazione tra mamme, Mamma 1 chiede come stanno andando a scuola i loro bambini. Mamma 2 condivide che Marco sta avendo difficoltà in matematica, ma stanno lavorando con un tutor. Mamma 3 dice che Laura ama la matematica, ma trova noiosa la storia. Mamma 4 menziona che anche Emma ha difficoltà con le date di storia, ma è brava in italiano. Infine, Mamma 1 nota che Pietro è bravo in tutte le materie, ma è molto disordinato e fatica a tenere in ordine il suo zaino e i quaderni.'),
 Document(id='6888178f-9fc0-4255-ade2-c2994c7844e6', metadata={'doc_id': '09b781b2-4fe0-44c2-9cc0-81ac85399ecf'}, page_content="Le mamme discutono della nuova maestra di inglese, che sembra essere molto competente. Una mamma menziona l'entusiasmo della sua figlia per le lezioni stimolanti, mentre un'altra spera che la nuova maestra possa aiutare sua figlia Emma, che ha sempr

## Query Ipotetiche

In [13]:
from pydantic import BaseModel
from typing import List
from langchain_core.runnables import RunnableLambda

class HypotheticalQuestions(BaseModel):
    """Generate hypothetical questions"""

    questions: List[str]

only_questions_parser = RunnableLambda(lambda x: x.questions)

chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template(
        "Generate a list of exactly 3 hypothetical questions that the below document could be used to answer:\n\n{doc}"
    )
    | ChatOpenAI(model="gpt-4o-mini", max_retries=3).with_structured_output(HypotheticalQuestions)
    | only_questions_parser
)

In [14]:
chain.invoke(docs[0])

['What would happen if the customer wanted to try on a shirt instead of pants?',
 "How would the interaction change if the customer didn't know their size?",
 'What might the conversation look like if the customer wanted a different color or style of pants?']

In [15]:
hypothetical_questions = chain.batch(docs, {"max_concurrency": 15})

In [16]:
hypothetical_questions

[['What would happen if the customer asked for a different color of the pants?',
  'How might the interaction change if the customer were looking for a shirt instead of pants?',
  'What if the customer had a sizing question about the pants they tried on?'],
 ['What would happen if the heat lasted for a whole week without any rain?',
  "How might the customer's plans in the city change if it rains again unexpectedly later in the day?",
  'What could be done to prepare for sudden summer storms like the one experienced by the customer?'],
 ['What if the Lakers had won the game instead of the Celtics?',
  'How would the conversation change if they were discussing a different sport, like soccer?',
  'What might have been the outcome if LeBron had performed at his usual level during the game?'],
 ['What might happen if the game had gone into double overtime?',
  'How would the conversation change if they were discussing a different sporting event instead?',
  "What could have been the reason

In [17]:
vectorstore = Chroma(collection_name="hypo-questions", embedding_function=OpenAIEmbeddings())

store = InMemoryByteStore()
id_key = "doc_id"

retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

doc_ids = [str(uuid.uuid4()) for _ in docs]

In [18]:
question_docs = []
for i, question_list in enumerate(hypothetical_questions):
    question_docs.extend(
        [Document(page_content=s, metadata={id_key: doc_ids[i]}) for s in question_list]
    )

In [19]:
question_docs

[Document(metadata={'doc_id': '915d1cb2-86df-4a27-b92b-1263f58b3275'}, page_content='What would happen if the customer asked for a different color of the pants?'),
 Document(metadata={'doc_id': '915d1cb2-86df-4a27-b92b-1263f58b3275'}, page_content='How might the interaction change if the customer were looking for a shirt instead of pants?'),
 Document(metadata={'doc_id': '915d1cb2-86df-4a27-b92b-1263f58b3275'}, page_content='What if the customer had a sizing question about the pants they tried on?'),
 Document(metadata={'doc_id': 'b8d052cd-2d90-4db7-b848-5e5b6942513c'}, page_content='What would happen if the heat lasted for a whole week without any rain?'),
 Document(metadata={'doc_id': 'b8d052cd-2d90-4db7-b848-5e5b6942513c'}, page_content="How might the customer's plans in the city change if it rains again unexpectedly later in the day?"),
 Document(metadata={'doc_id': 'b8d052cd-2d90-4db7-b848-5e5b6942513c'}, page_content='What could be done to prepare for sudden summer storms like th

In [20]:
retriever.vectorstore.add_documents(question_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [21]:
sub_docs = vectorstore.similarity_search("mamme che parlano")

In [22]:
sub_docs

[Document(id='5a765e58-8fba-4c73-a5ec-fb3815a52f63', metadata={'doc_id': '3c3229c9-4649-49d2-b9f9-fae8b3da7213'}, page_content='How would the conversation change if one of the mothers had a particularly good week?'),
 Document(id='9b01993c-7689-42c4-9ecb-f604a8438031', metadata={'doc_id': '3c3229c9-4649-49d2-b9f9-fae8b3da7213'}, page_content='What could the mothers do together to alleviate their fatigue and stress?'),
 Document(id='9b456155-1d87-4487-b532-fd3024052de0', metadata={'doc_id': '96d599b4-4ce8-4c51-bee3-131b196a46ef'}, page_content='What strategies do mothers use to manage childcare when their husbands are busy with work?'),
 Document(id='079e358b-f0a5-46a6-971e-b46da69ff241', metadata={'doc_id': '3c3229c9-4649-49d2-b9f9-fae8b3da7213'}, page_content='What might happen if the mothers decided to take a day off for themselves?')]

In [23]:
retrieved_docs = retriever.invoke("mamme che parlano")

In [24]:
for retr in retrieved_docs:
    print(retr, "\n")

page_content='Davanti alla scuola
Mamma 1 - Ciao ragazze! Come state oggi?
Mamma 2 - Ciao! Bene, grazie. Solo un po' stanca, Ã¨ stata una settimana lunga. E voi?
Mamma 3 - Anche io sono un po' stanca. Tra lavoro e figli, non ho un attimo di pausa!
Mamma 4 - Vi capisco perfettamente. Ogni tanto vorrei una giornata solo per me.' metadata={'source': 'data/dialoghi/dialogo3.txt'} 

page_content='Mamma 3 - E i vostri mariti? Come si stanno organizzando con i bambini?
Mamma 4 - Mio marito Ã¨ molto impegnato con il lavoro, ma cerchiamo di trovare sempre del tempo da passare insieme nel fine settimana.
Mamma 1 - Il mio cerca di aiutare il piÃ¹ possibile, anche se a volte sembra piÃ¹ un altro bambino da gestire!
Mamma 2 - Il mio Ã¨ molto coinvolto. Si occupa spesso dei compiti dei bambini, soprattutto in materie che io non capisco!
Mamma 3 - Il mio Ã¨ simile. Fa il possibile per essere presente. Siamo fortunate ad avere mariti collaborativi.' metadata={'source': 'data/dialoghi/dialogo3.txt'} 

